<a id="title_ID"></a>
# JWST Data Products: Calibrated Individual Exposures and WCS
--------------------------------------------------------------
**Author**: Alicia Canipe (acanipe@stsci.edu) with exerpts from Espinoza, Sosey | **Latest update**: March 30, 2021.

## Table of contents
1. [Introduction](#intro)
   1. [Resources](#resources)   
2. [Data in MAST](#mast)
3. [Example data for this exercise](#example)
4. [Data products: stage 1 (detector corrections)](#stage1)
    1. [Input](#s1-input)
    2. [Output](#s1-output)
    3. [Examining the products](#s1-examine)
5. [Associations](#associations)
6. [Data products: stage 2 (calibrated exposures)](#stage2)
    1. [Imaging](#s2-imaging)
        1. [Input](#s2-imaging-input)
        2. [Output](#s2-imaging-output)
    2. [Spectroscopy](#s2-spectroscopy)
        1. [Input](#s2-spectroscopy-input)
        2. [Output](#s2-spectroscopy-output)
7. [WCS deep dive](#wcs)
8. [Exercise](#exercise)

1.<font color='white'>-</font>Introduction <a class="anchor" id="intro"></a>
------------------

Welcome to the second module about JWST data products! In this module, we will take a deeper dive into how JWST data products change through different stages of the JWST Calibration Pipeline (hereafter, the pipeline). We will start by examining the data products for Stage 1 and Stage 2 processing, and end by checking out the WCS information. In the third and final module for this JWebbinar, we'll continue on to explore Stage 3 data products. 

### A.<font color='white'>-</font>Resources<a class="anchor" id="resources"></a>

Visit the [webpage for JWebbinars](https://www.stsci.edu/jwst/science-execution/jwebbinars) to find resources for:
* The Mikulski Archive for Space Telescopes (MAST) 
* JWST Documentation (JDox) for JWST data products
* The most up-to-date information about JWST data products in the pipeline readthedocs
* Pipeline roadmaps for when to recalibrate your data

Before we begin, import the libraries used in this notebook:

In [ ]:
# Module with functions to get information about objects:
import os
import inspect
import asdf 
import pprint

# Numpy library:
import numpy as np

# Scipy tools
from scipy.signal import medfilt

# Astropy tools:
from astropy.utils.data import download_file
from astropy.io import fits

# The JWST models:
from jwst import datamodels

And set up matplotlib for plotting:

In [ ]:
import matplotlib.pyplot as plt
import matplotlib as mpl

# Use this version if you want interactive plots
# %matplotlib notebook

# Use this version for non-interactive plots (easier scrolling of the notebook)
%matplotlib inline

# These gymnastics are needed to make the sizes of the figures
# be the same in both the inline and notebook versions
%config InlineBackend.print_figure_kwargs = {'bbox_inches': None}

mpl.rcParams['savefig.dpi'] = 80
mpl.rcParams['figure.dpi'] = 80

And finally, create some convenience functions. 

In [ ]:
def create_image(data_2d, vmin, vmax, xpixel=None, ypixel=None, title=None):
    ''' Function to generate a 2D image of the data, 
    with an option to highlight a specific pixel.
    '''
    
    fig = plt.figure(figsize=(8, 8))
    ax = plt.subplot()
    plt.imshow(data_2d, origin='lower', cmap='gray', vmin=vmin, vmax=vmax)
    
    if xpixel and ypixel:
        plt.plot(xpixel, ypixel, marker='o', color='red', label='Selected Pixel')

    plt.xlabel('Pixel column')
    plt.ylabel('Pixel row')
    
    if title:
        plt.title(title)

    fig.tight_layout()
    plt.subplots_adjust(left=0.15)
    plt.colorbar(label='DN')

In [ ]:
def create_slit_image(data_2d, slit_number, vmin=None, vmax=None, title=None):
    ''' Function to generate a 2D image of a particular slit.
    '''
    
    fig = plt.figure(figsize=(8, 8))
    ax = plt.subplot()
    
    if vmin and vmax:
        plt.imshow(data_2d.slits[slit_number].data, origin='lower', cmap='gray', vmin=vmin, vmax=vmax)
        plt.colorbar(label='DN/sec')        
    else:
        minimum = data_2d.slits[slit_number].data.min()
        maximum = data_2d.slits[slit_number].data.max()
        plt.imshow(data_2d.slits[slit_number].data, origin='lower', cmap='gray', vmin=minimum, vmax=maximum)
        plt.colorbar(label='DN/sec')        

    plt.xlabel('Pixel column')
    plt.ylabel('Pixel row')
    
    if title:
        plt.title(title)
    else:
        title = "object {} order = {}".format(data_2d.slits[slit_number].source_id, 
                                              data_2d.slits[slit_number].meta.wcsinfo.spectral_order)
        plt.title(title)

    plt.subplots_adjust(left=0.15)

In [ ]:
def plot_ramp(groups, signal, xpixel=None, ypixel=None, title=None):
    ''' Function to generate the ramp for pixel.
    '''
    
    fig = plt.figure(figsize=(8, 8))
    ax = plt.subplot()
    if xpixel and ypixel:
            plt.plot(groups, signal, marker='o', label='Pixel ('+str(xpixel)+','+str(ypixel)+')') 
            plt.legend(loc=2)

    else:
        plt.plot(groups, signal, marker='o')
        
    plt.xlabel('Groups')
    plt.ylabel('Signal (DN)')
    plt.subplots_adjust(left=0.15)
    
    if title:
        plt.title(title)

In [ ]:
def plot_column(data_2d, column, title=None):
    ''' Function to generate a plot for one column in a dispersed image.
    '''
    
    fig = plt.figure(figsize=(10, 5))
    ax = plt.subplot()
    plt.plot(data_2d[:,column], label='Column '+str(column))
        
    plt.xlabel('Pixel row')
    plt.ylabel('Column values')
    plt.subplots_adjust(left=0.15)
    
    if title:
        plt.title(title)
    else:
        plt.title('WFSS plot of one column in dispersed image')
        
    plt.legend()

In [ ]:
def plot_spectra(spec, number, median_filter=None, title=None):
    ''' Function to generate the spectrum for a slit.
    '''
    
    fig = plt.figure(figsize=(10, 5))
    ax = plt.subplot()
    
    if median_filter:
        plt.plot(spec.spec[number].spec_table['WAVELENGTH'], medfilt(spec.spec[number].spec_table['FLUX'],median_filter)) 
        
    else: 
        plt.plot(spec.spec[number].spec_table['WAVELENGTH'], spec.spec[number].spec_table['FLUX']) 

    
    plt.xlabel('Wavelength (um)')
    plt.ylabel('Flux')
    
    plt.subplots_adjust(left=0.15)
    
    if title:
        plt.title(title)
    else:
        title='Spectrum for Source '+str(spec.spec[number].source_id)+', Spectral Order '+str(spec.spec[number].spectral_order)
        plt.title(title)

[Top of Page](#title_ID)

2.<font color='white'>-</font>Data in MAST <a class="anchor" id="mast"></a>
------------------

The JWST Data Management System (DMS) produces many products for each JWST observation, including the science files generated by the pipeline. The exact type and number of products depends on the instrument, its configuration, and observing mode. Observers should consult the [MAST documentation for information about standard data products](https://jwst-docs.stsci.edu/obtaining-data/data-discovery#DataDiscovery-Dataproducttypes). 

Of the many different data products produced by the calibration pipeline, most observers will find the science data files in MAST to be sufficient for their analysis. However, other data products such as guide star data, associations, and engineering data are also available. 

Standard science data files include:

* [uncalibrated raw data](https://jwst-pipeline.readthedocs.io/en/stable/jwst/data_products/science_products.html#uncalibrated-raw-data-uncal), identified by the suffix ```uncal```
* [countrate data](https://jwst-pipeline.readthedocs.io/en/stable/jwst/data_products/science_products.html#countrate-data-rate-and-rateints) produced by applying the Stage 1 (detector-level) corrections in order to compute count rates from the original accumulating signal ramps, identified by the suffix ```rate``` or ```rateints```
* [calibrated single exposures](https://jwst-pipeline.readthedocs.io/en/stable/jwst/data_products/science_products.html#calibrated-data-cal-and-calints), identified by the suffix ```cal```
* [resampled and/or combined exposures](https://jwst-pipeline.readthedocs.io/en/stable/jwst/data_products/science_products.html#resampled-2-d-data-i2d-and-s2d), identified by the suffixes ```i2d``` or ```s2d```
* [extracted spectroscopic 1D data](https://jwst-pipeline.readthedocs.io/en/stable/jwst/data_products/science_products.html#extracted-1-d-spectroscopic-data-x1d-and-x1dints), identified by the suffixes ```x1d``` or ```c1d```

In addition, there are also [several other products depending on the observing mode](https://jwst-pipeline.readthedocs.io/en/stable/jwst/data_products/science_products.html#source-catalog-cat), such as source and photometry catalogs, stacked PSF data, and NIRISS AMI derived data.  

[Top of Page](#title_ID)

3.<font color='white'>-</font>Example data for this exercise <a class="anchor" id="example"></a>
------------------

For this module, we will use calibrated NIRCam simulated imaging and wide field slitless spectroscopy (WFSS) exposures that are stored in Box. Let's grab the data:

In [ ]:
rate_file = ["https://stsci.box.com/shared/static/h30hhwhu4ihlhqjnlhbblx07wnitoytd.fits", 
             "example_nircam_imaging_rate.fits"]
rateints_file = ["https://stsci.box.com/shared/static/jh937bjqodqhfobhpemnbqt4jax6d6j4.fits", 
                 "example_nircam_imaging_rateints.fits"]
ramp_file = ["https://stsci.box.com/shared/static/x7d0ldm7bp683p5yyi2buvphjcckujbe.fits",
             "example_nircam_imaging_ramp.fits"]
wfss_rate_file = ["https://stsci.box.com/shared/static/d5k9z5j05dgfv6ljgie483w21kmpevni.fits",
                  "example_nircam_wfss_rate.fits"]
cal_file = ["https://stsci.box.com/shared/static/8g15cxb3nri47l3bx22mjtdw3yt8xxiv.fits",
            "example_nircam_imaging_cal.fits"]
wfss_cal_file = ["https://stsci.box.com/shared/static/pqgt98wsjz16av3768756ierahzqn8w7.fits",
                 "example_nircam_wfss_cal.fits"]
wfss_x1d_file = ["https://stsci.box.com/shared/static/fjzq3dm2kgp2ttoptxwe9yfghmxxxz89.fits",
                 "example_nircam_wfss_x1d.fits"]
demo_ex_file = ["https://stsci.box.com/shared/static/6vn402728z12cyx6czdt5hpaxa071aek.fits",
                "example_exercise_cal.fits"]

all_files = [rate_file, rateints_file, ramp_file, cal_file,
             wfss_rate_file, wfss_cal_file, wfss_x1d_file,
             demo_ex_file]

for file in all_files:
    demo_file = download_file(file[0])
    
    # Save the file so that we can use it later
    with fits.open(demo_file) as f:
        f.writeto(file[1], overwrite=True)

[Top of Page](#title_ID)

4.<font color='white'>-</font>Data products: stage 1 (detector corrections)<a class="anchor" id="stage1"></a>
------------------

All JWST data, regardless of the instrument or mode (with the exception of a few specific engineering or calibration cases), is processed through the CALWEBB_DETECTOR1 module, which is Stage 1 of the pipeline. A number of instrument signatures are accounted for in this stage, such as bias corrections and cosmic ray flagging, and slopes are fit to the corrected ramps. More information can be found in the [JWST User Documentation for CALWEBB_DETECTOR1](https://jwst-docs.stsci.edu/jwst-data-reduction-pipeline/algorithm-documentation/stages-of-processing/calwebb_detector1). We also have a full list of data product types and the units of the data for each product [in the documentation](https://jwst-pipeline.readthedocs.io/en/latest/jwst/data_products/product_types.html#data-product-types). 

### A.<font color='white'>-</font>Input<a class="anchor" id="s1-input"></a>

The inputs to this stage are listed below.

* **4D raw data**
    * **Data model**: RampModel. 
    * **File suffix**: ```_uncal```
    * **Description**: A single raw 4D exposure that contains the original raw data from all detector readouts in the exposure (ncols x nrows x ngroups x nintegrations) 



### B.<font color='white'>-</font>Output<a class="anchor" id="s1-output"></a>

The outputs of this stage are listed below.

* **2D countrate product**
    * **Data model**: ImageModel or IFUImageModel
    * **File suffix**: ```_rate```
    * **Description**: All types of inputs result in a 2D countrate product, containing the average over all integrations in an exposure. This product is passed along to subsequent pipeline modules for all non-time series and non-coronagraphic exposures. For MIRI MRS and NIRSpec IFU, the output data model will be IFUImageModel, while all others will be ImageModel.
    
    
* **3D countrate product**
    * Data model: CubeModel
    * File suffix: ```_rateints```
    * Description: A 3D countrate product is created that contains the individual results of each integration. The 2D countrate images for each integration are stacked along the 3rd axis of the data cubes (ncols x nrows x nints). The 3D ```_rateints``` product is passed along to pipeline modules for all time series and coronagraphic exposures. 
    
   
* (optional) **4D corrected ramp**
    * **Data model**: RampModel
    * **File suffix**: ```_ramp```
    * **Description**: Result of applying all pipeline steps up through the jump (cosmic ray detection) step, to produce corrected and cosmic ray-flagged 4D ramp data, which will have the same data dimensions as the input raw 4D data (ncols x nrows x ngroups x nints). This file is not created by default, so a user must manually run the pipeline and set the argument ```--save_calibrated_ramp``` to True. 

### C.<font color='white'>-</font>Examining the products<a class="anchor" id="s1-examine"></a>

Let's take a closer look at some data products. As our first example, we can grab the ```rate``` and ```rateints``` data products for the simulation we used in module 1. Looking above, we can see that these types of data use the ImageModel and the CubeModel, respectively.

In [ ]:
# Load the data into models (use: rate_image, rateints_cube)
rate_image = datamodels.ImageModel(rate_file[1])
rateints_cube = datamodels.CubeModel(rateints_file[1])

Remember that the models are structured this way:

In [ ]:
# Check out the structure of the rate file
rate_image.info()

Take note of the arrays that we have now:
* ```data```
* ```dq```
* ```err```
* ```var_poisson```
* ```var_rnoise```

These can be accessed the same way we described before:

In [ ]:
# Print the shape of the science data array for the rate image
rate_image.data.shape

In [ ]:
# Print the shape of the science data array for the rateints image
rateints_cube.data.shape

In [ ]:
# Grab the variance due to poisson noise 
variance_poisson = rate_image.var_poisson
variance_poisson

In [ ]:
# Create an image of the rate data
create_image(rate_image.data, 0, 10, title="2D image data product")

In [ ]:
# Compare that to an image of one integration for the rateints data
create_image(rateints_cube.data[-1,:,:], 0, 10, title="3D cube data product")

We can also examine the ramp for a pixel with the bias drift removed using the optional 4D ```_ramp.fits``` file to revisit the up-the-ramp sampling, with detector corrections applied: 

In [ ]:
# Use: integration, pixel_y, pixel_x, group
integration = 0
pixel_y = 741
pixel_x = 1798
group = -1

In [ ]:
# Load the ramp_file into the RampModel and set up arrays to plot (use: ramp_model, groups, signal_adu)
ramp_model = datamodels.RampModel(ramp_file[1])
groups = np.arange(0, ramp_model.meta.exposure.ngroups)
signal_adu = ramp_model.data[integration, :, pixel_y, pixel_x]

In [ ]:
# Plot the ramp
plot_ramp(groups, signal_adu, xpixel=pixel_x, ypixel=pixel_y, title="Optional ramp data product")

Let's take a look at the metadata for our output products, but rather than using the standard FITS methods, let's use the data model to access the information. 

The metadata has been updated after going through Stage 1 processing: 

In [ ]:
# Look at the full metadata list
rate_image.meta.instance

Make note of a few things in the metadata. ```cal_step``` shows you which pipeline steps were applied. You also have the particular calibration reference files that were used for each step, shown in the ```ref_file``` entries. Alternately, you can search the tree for particular keywords:

In [ ]:
# Find datamodel equivalent of the FITS keyword indicating that the linearity correction was done (S_LINEAR)
rate_image.find_fits_keyword('S_LINEAR') 

In [ ]:
# Search the datamodel for information related to units 
rate_image.search_schema('unit')

In [ ]:
# Print the data unit
rate_image.meta.bunit_data

Now, let's take a look at some spectroscopic data -- how about a NIRCam WFSS dispersed image? At this stage, the structure will be roughly the same as for our other image example.

In [ ]:
# Load the WFSS image into the appropriate model (use: wfss_image)
wfss_image = datamodels.ImageModel(wfss_rate_file[1])

In [ ]:
# Create an image of the data
create_image(wfss_image.data, vmin=-0.05, vmax=0.5, title="2D WFSS data before Stage 2")

In [ ]:
# Look at the structure of the model
wfss_image.info()

[Top of Page](#title_ID)

5.<font color='white'>-</font>Associations<a class="anchor" id="associations"></a>
------------------

Now that we're moving on to examine data products for Stage 2 and Stage 3 processing, it would be a good time to mention JWST associations, since the association files are a part of the JWST data products used to process data through Stage 2 and Stage 3. Associations are basically just lists of files, mostly exposures, that are related in some way. For JWST, associations have the following characteristics:

* Relationships between multiple exposures are captured in an association.
* An association is a means of identifying a set of exposures that belong together and may be dependent upon one another.
* The association concept permits exposures to be calibrated, archived, retrieved, and reprocessed as a set rather than as individual objects.

In general, it takes many exposures to make up a single observation, and an entire program is made up of a large number of observations. Given a set of exposures for a program, there is a tool that groups the exposures into individual associations. These associations are then used as input to the Stage 2 and 3 calibration steps to perform the transformation from exposure-based data to source-based, high(er) signal-to-noise data. The association used to process data is available in MAST as part of the "Info" data product category. You can read more about associations [here](https://jwst-pipeline.readthedocs.io/en/latest/jwst/associations/index.html). 

An example of a Stage 2 association is shown [in the documentation](https://jwst-pipeline.readthedocs.io/en/latest/jwst/associations/level2_asn_technical.html#example-association), along with a [Stage 3 association](https://jwst-pipeline.readthedocs.io/en/latest/jwst/associations/level3_asn_technical.html#example-association). Unless you are generating your own data or simulations, you will probably not need to create an association file, because you will have the option to retrieve association files from MAST along with your data for reprocessing.  

However, if you do want to create an association, there are also [command line tools](https://jwst-pipeline.readthedocs.io/en/latest/jwst/associations/asn_from_list.html) included in the pipeline software that help with generating associations for manually running the pipeline. 

Now that that's out of the way, let's continue our data products journey. 

[Top of Page](#title_ID)

6.<font color='white'>-</font>Data products: stage 2 (calibrated exposures)<a class="anchor" id="stage2"></a>
------------------

The paths through the pipeline begin to diverge during Stage 2 for different observing modes. This stage applies physical corrections and calibrations to individual exposures to produce fully calibrated (unrectified) exposures, and the pipeline module used depends on the exposure type: either imaging or spectroscopy. More information can be found in the [JWST User Documentation](https://jwst-docs.stsci.edu/jwst-data-reduction-pipeline/algorithm-documentation/stages-of-processing). We also have a full list of data product types and the units of the data for each product [in the documentation](https://jwst-pipeline.readthedocs.io/en/latest/jwst/data_products/product_types.html#data-product-types). 

## 6.1.<font color='white'>-</font>Imaging<a class="anchor" id="s2-imaging"></a>

Stage 2 image processing applies additional instrumental corrections and calibrations that result in a fully calibrated individual exposure. Non-time series exposures use the CALWEBB_IMAGE2 module, which applies all applicable steps to the data. The CALWEBB_TSO-IMAGE2 module, on the other hand, should be used for time series exposures, for which some steps are set to be skipped by default. Both modules call the Image2Pipeline; the only difference is which steps are applied.

### A.<font color='white'>-</font>Input<a class="anchor" id="s2-imaging-input"></a>

The inputs to this stage are listed below.

* **2D or 3D countrate data**
    * **Data model**: ImageModel or CubeModel
    * **File suffix**: ```_rate``` or ```_rateints```
    * **Description**: The input to Image2Pipeline is a countrate exposure, either ```_rate``` or ```_rateints``` data. A single input file can be processed or an association file listing multiple inputs can be used, in which case the processing steps will be applied to each input exposure, one at a time. If ```_rateints``` products are used, each step applies its algorithm to each integration in the exposure. Time series and coronagraphic exposures are expected to use 3D data as input, to be processed on a per-integration basis. Time series exposures will use the CALWEBB_TSO2 module, while coronagraphic exposures should use the CALWEBB_IMAGE2 module.
    


### B.<font color='white'>-</font>Output<a class="anchor" id="s2-imaging-output"></a>

The outputs of this stage are listed below.

* **2D or 3D calibrated data**
    * **Data model**: ImageModel or CubeModel
    * **File suffix**: ```_cal``` or ```_calints```
    * **Description**: The output is a fully calibrated, but unrectified, exposure, using the product type suffix ```_cal``` or ```_calints```, depending on the type of input.
    
    
* **2D resampled image**
    * Data model: DrizProductModel
    * File suffix: ```_i2d```
    * Description: This is the output of the resample step and is only created for regular direct imaging observations (not for time series or coronagraphy 3D data sets). Note that this product is intended for quick-look use only and is not passed along as input to Stage 3 processing. Calibrated, but unrectified (```_cal```) products are used as input to Stage 3. Here, "resampled" just means that rectified 2D images are created using the AstroDrizzle algorithm and the attached WCS information.
    
   
* (optional) **2D or 3D background-subtracted data**
    * **Data model**: ImageModel or CubeModel
    * **File suffix**: ```_bsub``` or ```_bsubints```
    * **Description**: This is an intermediate product that is only created if the ```–save_bsub``` parameter is set to True. It will contain the data as output from the background step. If the input is a ```_rate``` product, this will be a ```_bsub``` product, while ```_rateints``` inputs will be saved as ```_bsubints```.  



Let's take a quick look at the standard imaging data product from Stage 2 that is used as input to Stage 3 processing: 

In [ ]:
# Load the calibrated image into a model (use: cal_image)
cal_image = datamodels.ImageModel(cal_file[1])

In [ ]:
# Plot the image
create_image(cal_image.data[:,:], 0, 10, title="2D calibrated image")

In [ ]:
# Check out the model structure
cal_image.info()

Notice the arrays that are available in the calibrated images above:

* ```area```
* ```data```
* ```dq```
* ```error```
* ```var_flat```
* ```var_poisson```
* ```var_rnoise```

Also notice the ```bunit_data``` and ```bunit_err``` metadata values - those provide the units for the data. The metadata and data arrays can be accessed in the way we described before:

In [ ]:
# Grab the flat variance array
variance_flat = cal_image.var_flat

In [ ]:
# Print the units
cal_image.meta.bunit_data

You'll notice in the metadata that there is more information -- for example, the association file name, data units, and WCS information. We'll revisit the WCS in the last section of this module. 

In [ ]:
# Check out the entire metadata list
cal_image.meta.instance

## 6.2.<font color='white'>-</font>Spectroscopy<a class="anchor" id="s2-spectroscopy"></a>

Stage 2 spectroscopic processing applies additional instrumental corrections and calibrations to countrate products that result in a fully calibrated individual exposures. There are two unique configurations (meaning, the steps applied and the order they are applied in) used to control this pipeline, depending on whether the data are to be treated as time series observations. Non-time series exposures use the CALWEBB_SPEC2 configuration, which applies all applicable steps to the data. The CALWEBB_TSO-SPEC2 configuration, on the other hand, should be used for time series exposures, which skips some steps by default. Both configurations call the Spec2Pipeline module; the only difference is which steps are applied.

The Spec2Pipeline is the “Swiss army knife” of pipeline modules, containing many steps that are only applied to certain instruments or instrument modes. The logic for determining which steps are appropriate is built into the pipeline module itself and is mostly based on either the instrument name or the exposure type (EXP_TYPE keyword) of the data.

### A.<font color='white'>-</font>Input<a class="anchor" id="s2-spectroscopy-input"></a>

The inputs to this stage are listed below.

* **2D or 3D countrate data**
    * **Data model**: ImageModel, IFUImageModel, or CubeModel
    * **File suffix**: ```_rate``` or ```_rateints```
    * **Description**: The input to the Spec2Pipeline pipeline is a countrate exposure, either ```_rate``` or ```_rateints``` data. A single input file can be processed or an association file listing multiple inputs can be used, in which case the processing steps will be applied to each input exposure, one at a time. If ```_rateints``` products are used as input, each step applies its algorithm to each integration in the exposure. Note that some steps can only be executed when the pipeline is given an association file as input, because they rely on multiple, associated exposures to perform their tasks. The association file must list not only the input science exposure(s), but must also list the exposures to be used as background or imprint. The input data model type IFUImageModel is only used for MIRI MRS and NIRSpec IFU exposures.
    

### B.<font color='white'>-</font>Output<a class="anchor" id="s2-spectroscopy-output"></a>

The outputs of this stage are listed below.

* **2D or 3D calibrated data**
    * **Data model**: ImageModel, IFUImageModel, CubeModel, SlitModel, or MultiSlitModel
    * **File suffix**: ```_cal``` or ```_calints```
    * **Description**: The output is a fully calibrated, but unrectified, exposure, using the product type suffix ```_cal``` or ```_calints```, dependening on the type of input.The output data model type can be any of the 4 listed above and is completely dependent on the type of input data and the observing mode. For data sets that do not go through 2D spectral extraction, the output will be either a ImageModel, IFUImageModel, or CubeModel, matching the corresponding input data type. Of the data types that do go through extract_2d processing, the output type will consist of either a single slit model or a mutli-slit model:

* NIRSpec Bright-Object and NIRCam TSO Grism: SlitModel
* NIRSpec Fixed Slit and MOS, as well as WFSS: MultiSlitModel

The multi-slit model is an array of multiple slit models, each one containing the data and relevant metadata for a particular extracted slit or source. A MultiSlitModel product will contain multiple tuples of SCI, ERR, DQ, WAVELENGTH, etc. arrays; one for each of the extracted slits/sources.
    
    
* **2D resampled data**
    * Data model: DrizProductModel or MultiProductModel
    * File suffix: ```_s2d```
    * Description: If the input is a 2D exposure type that gets resampled/rectified, the rectified 2D spectral product is saved as a ```_s2d``` file. This image is intended for use as a quick-look product only and is not used in subsequent processing. The 2D unresampled, calibrated (```_cal```) products are passed along as input to subsequent Stage 3 processing. If the input is a MultiSlitModel, then the resampled output will be in the form of a MultiProductModel, which contains an array of individual models, one per slit. Otherwise the output will be a DrizProductModel. Here, "resampled" just means that rectified 2D images are created using the AstroDrizzle algorithm and the attached WCS information.
    
    
* **3D resampled (IFU cube) data**
    * Data model: IFUCubeModel
    * File suffix: ```_s3d```
    * Description: If the data are NIRSpec IFU or MIRI MRS, the result of the cube building step will be a 3D IFU spectroscopic cube saved to a ```_s3d``` file. The IFU cube is built from the data contained in a single exposure and is intended for use as a quick-look product only and is not used in subsequent processing. The 2D unresampled, calibrated (```_cal```) products are passed along as input to subsequent Stage 3 processing.
    
    
* **1D extracted spectral data**
    * Data model: MultiSpecModel
    * File suffix: ```_x1d``` or ```_x1dints```
    * Description: All types of inputs result in a 1D extracted spectral data product, which is saved as a ```_x1d``` or ```_x1dints``` file, depending on the input type. Observing modes such as MIRI LRS fixed slit and MRS, NIRCam and NIRISS WFSS, and NIRSpec fixed slit, MOS, and IFU result in an ```_x1d``` product containing extracted spectral data for one or more slits/sources. Time series modes, such as MIRI LRS slitless, NIRCam time series grism, NIRISS SOSS, and NIRSpec Bright Object, for which the data are 3D stacks of integrations, result in ```_x1dints``` products containing extracted spectral data for each integration with the exposure.
    
   
* (optional) **2D or 3D background-subtracted data**
    * **Data model**: ImageModel, IFUImageModel, or CubeModel
    * **File suffix**: ```_bsub``` or ```_bsubints```
    * **Description**: This is an intermediate product that is only created if the ```–save_bsub``` parameter is set to True. It will contain the data as output from the background step. If the input is a ```_rate``` product, this will be a ```_bsub``` product, while ```_rateints``` inputs will be saved as ```_bsubints```.  

Let's revisit our NIRCam WFSS data from before, but this time we'll look at the ```_cal.fits``` calibrated output from Stage 2 that is used as input to Stage 3. It's going to look a little different. 

In [ ]:
# Load the calibrated WFSS data into a model (use: cal_wfss) 
cal_wfss = datamodels.MultiSpecModel(wfss_cal_file[1])

In [ ]:
# Check out the structure
cal_wfss.info()

Here, we no longer have the ```data``` array, because the model contains extracted spectral data for one or more slits/sources.

In [ ]:
# What's in slits?
cal_wfss.slits

You can examine the information for the different slits, e.g., the photometric conversion used, pixel area, etc.):

In [ ]:
# Choose a slit, say slit #10, and check out all the meta data (use: slit_number)
slit_number = 12
cal_wfss.slits[slit_number].meta.instance

In [ ]:
# Print the source ID, spectral order, bounding box, source position, and data mean
print('\nSlit number: ', slit_number)
print('Source ID: ', cal_wfss.slits[slit_number].source_id)
print('Spectral order: ', cal_wfss.slits[slit_number].meta.wcsinfo.spectral_order)
print('Bounding box: ', cal_wfss.slits[slit_number].meta.wcs.bounding_box)
print('Source X position, Y position (full frame coordinates): ', cal_wfss.slits[slit_number].source_xpos,cal_wfss.slits[slit_number].source_ypos)
print('Data average: ', cal_wfss.slits[slit_number].data.mean())
print('\n')

In [ ]:
# Look at the WCS information for a particular column and row (use: column, row, ra, dec, wavelength, order)
column, row = 100, 4
ra, dec, wavelength, order = cal_wfss.slits[slit_number].meta.wcs(column, row)
print('RA: ', ra)
print('Dec: ', dec)
print('Wavelength: ',wavelength)
print('Order: ',order)

What does this slit look like? 

In [ ]:
# Create an image of this slit
create_slit_image(cal_wfss, slit_number, vmin=-0.1, vmax=0.3)

Or plot one column of the dispersed image for our slit:

In [ ]:
# Plot one column of the slit
plot_column(cal_wfss.slits[slit_number].data, column)

What about the 1D extracted spectral data product, the ```_x1d.fits``` file? At first glance using FITS, this file can appear very complicated because there is one extension for each source and spectral order:

In [ ]:
# Use FITS to examine the structure of this file 
with fits.open(wfss_x1d_file[1]) as h:
    h.info()
    for i in np.arange(1,len(h)-1):
        print('\nExtension: ',i)
        print('Source ID: ',h[i].header['SOURCEID'])                
        print('Spectral Order: ',h[i].header['SPORDER'])

However, the ```MultiSpecModel``` makes it much easier to work with this file:

In [ ]:
# Switch to a datamodel (use: spec)
spec = datamodels.MultiSpecModel(wfss_x1d_file[1])

In [ ]:
#What's the shape of spec.spec?
print(len(spec.spec))

We can choose our source and the spectral order, and plot the spectrum:

In [ ]:
# Get the source ID and spectral order, just use the same slit_number
print('Source ID: ', spec.spec[slit_number].source_id)
print('Spectral order: ', spec.spec[slit_number].spectral_order)

In [ ]:
# Plot the spectrum
plot_spectra(spec, slit_number, median_filter=11)      

[Top of Page](#title_ID)

7.<font color='white'>-</font>WCS deep dive<a class="anchor" id="wcs"></a>
------------------

The first step in Stage 2 processing ("Assign WCS") is where the information to transfer the pixel coordinates to astronomical coordinates (e.g., RA and Dec) is added to the data. The WCS information and distortion model are provided by instrument- and detector- specific calibration reference files. The data itself is not modified by this step, it just associates a WCS object with each science exposure. The WCS object transforms positions in the detector frame to positions in a world coordinate frame - ICRS and wavelength. In general, there may be intermediate coordinate frames depending on the instrument. The WCS is saved in the ASDF extension of the FITS file and can be accessed as an attribute of the meta object when the FITS file is opened as a data model.

#### From the FITS WCS standard to the Generalized World Coordinate System (GWCS)

The JWST Assign WCS step is based on [GWCS](https://gwcs.readthedocs.io/en/latest/) and uses the modeling, units and coordinates subpackages in Astropy. GWCS provides a more general approach to the problem of expressing transformations between pixel and world coordinates. It supports a data model that includes the entire transformation pipeline from input coordinates (detector by default) to world coordinates and is tightly integrated with Astropy, rather than using the FITS WCS standard, which only provides instructions on how to relate pixel to world. GWCS will provide observers with the complete transform. 

The forward direction of the transforms is from detector to world coordinates and the input positions are 0-based. Basic WCS keywords are contained in the science headers, while distortion and spectral models are stored in reference files in the ASDF format.

For each observing mode, the "Assign WCS" step (or ```assign_wcs```) retrieves calibration reference files and creates a pipeline of transforms from the input detector frame to the V2,V3 frame. This part of the WCS pipeline may include intermediate coordinate frames. The basic WCS keywords are used to create the transform from V2,V3 frame to world coordinates.

To display images with software like DS9 that relies on specific WCS information, a SIP-based approximation to the WCS is fit. The results are FITS keywords stored in ```model.meta.wcsinfo```. While this is not an exact fit, it is accurate to ~0.25 pixel and should be sufficient for display purposes. This step is performed by default, but observers can turn it off manually or adjust the parameters that control the fit. This step will be discussed more in the pipeline tutorial JWebbinars. 

Let's start with the imaging example to interact with the WCS.

After the Assign WCS step is run in Stage 2, a GWCS object that contains all the transforms is now attached to the image model. What does the GWCS WCS look like? 

In [ ]:
# Look at the WCS info in the calibrated image model 
cal_image.meta.wcs

How does this compare to the FITS WCS standard?

In [ ]:
# How does this compare to the FITS wcs? (use: image_fits_wcs)
image_fits_wcs = cal_image.get_fits_wcs()
image_fits_wcs

Compare that with the WCS information in the datamodel:

In [ ]:
# Look at CRVAL and CRPIX in the datamodel
print('CRVAL: ', cal_image.meta.wcsinfo.crval1, cal_image.meta.wcsinfo.crval2)
print('CRPIX: ', cal_image.meta.wcsinfo.crpix1, cal_image.meta.wcsinfo.crpix2)

You can see what coordinate frames are available to you:

In [ ]:
# What coordinate frames are available?
cal_image.meta.wcs.available_frames  

In [ ]:
# What was the input frame for this data?
cal_image.meta.wcs.input_frame

In [ ]:
# What was the output frame?
cal_image.meta.wcs.output_frame

We can check the transform from detector pixels to sky coordinates in decimal degrees of RA and Dec: 

In [ ]:
# Get the WCS info for pixel 110, 110
cal_image.meta.wcs(110, 110)

In [ ]:
# Get the transform to go from detector to world coordinates (use: detector_to_world)
detector_to_world = cal_image.meta.wcs.get_transform('detector', 'world')

In [ ]:
# Do the transformation for our pixel (use: pix_ra, pix_dec)
pix_ra, pix_dec = detector_to_world(110, 110)
pix_ra, pix_dec

And the inverse transform: 

In [ ]:
# Now get the reverse transform from world to detector (use: world_to_detector)
world_to_detector = cal_image.meta.wcs.get_transform('world', 'detector')

In [ ]:
# Do the transformation using our outputs from above
world_to_detector(pix_ra, pix_dec)

What about our spectral data? Let's take a look: 

In [ ]:
# What frames are available?
cal_wfss.slits[slit_number].meta.wcs.available_frames

In [ ]:
# What was the bounding box used for the cutout?
cal_wfss.slits[slit_number].meta.wcs.bounding_box

In [ ]:
# Look at CRVAL and CRPIX in the datamodel
print('CRVAL: ', cal_wfss.meta.wcsinfo.crval1, cal_wfss.meta.wcsinfo.crval2)
print('CRPIX: ', cal_wfss.meta.wcsinfo.crpix1, cal_wfss.meta.wcsinfo.crpix2)

In [ ]:
# Get the WCS info for multiple pixels: (200, 4) and (30, 3) -- (use: ra, dec, wavelength, order)
ra, dec, wavelength, order = cal_wfss.slits[slit_number].meta.wcs([10, 4], [6, 3])
ra, dec, wavelength, order

You can do the backwards transformation:

In [ ]:
# Get the world to detector transform (use: world_to_detector_ss)
world_to_detector_ss = cal_wfss.slits[slit_number].meta.wcs.get_transform('world','detector') 

In [ ]:
# And go from world to detector coordinates (use: x0, y0, wave, order2)
x0, y0, wave, order2 = world_to_detector_ss(ra, dec, wavelength, order)
print(x0, y0, wave, order2)

And the forward transformation:

In [ ]:
# Now, do the inverse (use: detector_to_world_ss)
detector_to_world_ss = cal_wfss.slits[slit_number].meta.wcs.get_transform('detector','world')
detector_to_world_ss(x0, y0, wave, order2)

The next and final module will discuss the Stage 3 data products, the last stage of processing in the pipeline.

[Top of Page](#title_ID)

8.<font color='white'>-</font>Exercise<a class="anchor" id="exercise"></a>
--------------------------------------------------------------------
Now, you try it!

In [ ]:
#Load the exercise data using a model
with datamodels.open(demo_ex_file[1]) as exercise_data:
    exercise_data.info()

In [ ]:
# What instrument and mode are used here?
exercise_data.meta.instrument.name, exercise_data.meta.exposure.type

In [ ]:
# What are the data units? 
exercise_data.meta.bunit_data

In [ ]:
# Which calibration steps were applied?
exercise_data.meta.cal_step.instance

In [ ]:
# Choose a pixel or pixels and get the WCS information
det_x, det_y = 10, 5
ra, dec, wave, order = exercise_data.slits[0].meta.wcs(det_x, det_y)
ra, dec, wave, order 

In [ ]:
# Now get the detector to world transform
d2w = exercise_data.slits[0].meta.wcs.get_transform('detector','world') 

In [ ]:
# Do the transformormation
ra, dec, wavelength, order = d2w(det_x, det_y, wave, order)

In [ ]:
# Now get the inverse transform
w2d = exercise_data.slits[0].meta.wcs.get_transform('world','detector') 

In [ ]:
# Do the transformation - do you get your pixel back? 
w2d(ra, dec, wavelength, order)

[Top of Page](#title_ID)